In [1]:
!pip install flask pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=ad3974dfc2463154fb5c78f8de3c04d46e49be48e21c5bb97a7e6d65cbc7cabd
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
!pip install pyngrok

In [3]:
!pip install flask flask-ngrok

In [4]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template

In [5]:
!pip install --upgrade pyngrok

In [6]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel

from pyspark.ml.classification import LogisticRegressionModel
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType



In [8]:
!pip install flask-cors

In [ ]:
from flask_cors import CORS
app = Flask(__name__)
CORS(app)
spark = SparkSession.builder \
    .appName("SentimentAnalysisAPI") \
    .master("local[*]") \
    .getOrCreate()

pipelineModel = PipelineModel.load("/content/drive/MyDrive/pipeline_folder")
lrModel = LogisticRegressionModel.load("/content/drive/MyDrive/model_folder")

def map_prediction(prediction):
    if prediction == 0:
        return "Negative"
    elif prediction == 2:
        return "Neutral"
    elif prediction == 1:
        return "Positive"
    else:
        return "Unknown"

map_prediction_udf = udf(map_prediction, StringType())

@app.route('/')
def home():
    return "Hello, World!"

@app.route('/predict', methods=['POST'])
def predict_sentiment():
    data = request.json

    if 'text' not in data:
        return jsonify({"error": "No text field provided"}), 400

    new_data = spark.createDataFrame([(1, data['text'])], ["id", "clean_text"])

    transformed_new_data = pipelineModel.transform(new_data)

    predictions = lrModel.transform(transformed_new_data)

    predictions_with_labels = predictions.withColumn("predicted_label", map_prediction_udf("prediction"))

    result = predictions_with_labels.select("predicted_label").collect()[0]["predicted_label"]

    return jsonify({"text": data['text'], "sentiment": result})

if __name__ == '__main__':
    ngrok.set_auth_token('2lC10VNMNNHozy9qU2wBzosN3at_3QYjZW2FJ2sr1po7qXqqs')

    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

    app.run(port=5000)


 * ngrok tunnel "NgrokTunnel: "https://7e65-35-227-169-165.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:17:04] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:17:11] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:17:27] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:17:30] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:17:38] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:17:41] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:17:47] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:17:49] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12:18:45] "OPTIONS /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2024 12